In [81]:
import pandas as pd 
import numpy as np

In [82]:
df_solar = pd.read_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/HEFTcom24/data/solar2.csv",)
df_solar_total = pd.read_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/basic_files/solar_total_production.csv")

In [83]:
df_solar = df_solar.drop(df_solar.columns[0], axis= 1)
#df_solar.sort_values(by= "valid_time")
#df_solar = df_solar.groupby("valid_time").tail(1).reset_index(drop= True)
df_solar["valid_time"] = pd.to_datetime(df_solar["valid_time"], utc= True)

In [84]:
df_solar_total.timestamp_utc = pd.to_datetime(df_solar_total.timestamp_utc)

In [85]:
df = pd.merge(df_solar_total, df_solar, left_on='timestamp_utc',right_on='valid_time', how='inner')

In [86]:
mask = (df["reference_time"] >= "2022-11-24") & (df["reference_time"] <= "2022-12-07")

dwd_cols = [
    'CloudCover_Point0_dwd', 'CloudCover_Point1_dwd', 'CloudCover_Point2_dwd',
    'CloudCover_Point3_dwd', 'CloudCover_Point4_dwd', 'CloudCover_Point5_dwd',
    'CloudCover_Point6_dwd', 'CloudCover_Point7_dwd', 'CloudCover_Point8_dwd',
    'CloudCover_Point9_dwd', 'CloudCover_Point10_dwd', 'CloudCover_Point11_dwd',
    'CloudCover_Point12_dwd', 'CloudCover_Point13_dwd', 'CloudCover_Point14_dwd',
    'CloudCover_Point15_dwd', 'CloudCover_Point16_dwd', 'CloudCover_Point17_dwd',
    'CloudCover_Point18_dwd', 'CloudCover_Point19_dwd', 'SolarDownwardRadiation_Point0_dwd',
    'SolarDownwardRadiation_Point1_dwd', 'SolarDownwardRadiation_Point2_dwd',
    'SolarDownwardRadiation_Point3_dwd', 'SolarDownwardRadiation_Point4_dwd',
    'SolarDownwardRadiation_Point5_dwd', 'SolarDownwardRadiation_Point6_dwd',
    'SolarDownwardRadiation_Point7_dwd', 'SolarDownwardRadiation_Point8_dwd',
    'SolarDownwardRadiation_Point9_dwd', 'SolarDownwardRadiation_Point10_dwd',
    'SolarDownwardRadiation_Point11_dwd', 'SolarDownwardRadiation_Point12_dwd',
    'SolarDownwardRadiation_Point13_dwd', 'SolarDownwardRadiation_Point14_dwd',
    'SolarDownwardRadiation_Point15_dwd', 'SolarDownwardRadiation_Point16_dwd',
    'SolarDownwardRadiation_Point17_dwd', 'SolarDownwardRadiation_Point18_dwd',
    'SolarDownwardRadiation_Point19_dwd', 'Temperature_Point0_dwd',
       'Temperature_Point1_dwd', 'Temperature_Point2_dwd',
       'Temperature_Point3_dwd', 'Temperature_Point4_dwd',
       'Temperature_Point5_dwd', 'Temperature_Point6_dwd',
       'Temperature_Point7_dwd', 'Temperature_Point8_dwd',
       'Temperature_Point9_dwd', 'Temperature_Point10_dwd',
       'Temperature_Point11_dwd', 'Temperature_Point12_dwd',
       'Temperature_Point13_dwd', 'Temperature_Point14_dwd',
       'Temperature_Point15_dwd', 'Temperature_Point16_dwd',
       'Temperature_Point17_dwd', 'Temperature_Point18_dwd',
       'Temperature_Point19_dwd', 'Temperature_Point0_ncep',
       'Temperature_Point1_ncep', 'Temperature_Point2_ncep',
       'Temperature_Point3_ncep', 'Temperature_Point4_ncep',
       'Temperature_Point5_ncep', 'Temperature_Point6_ncep',
       'Temperature_Point7_ncep', 'Temperature_Point8_ncep',
       'Temperature_Point9_ncep', 'Temperature_Point10_ncep',
       'Temperature_Point11_ncep', 'Temperature_Point12_ncep',
       'Temperature_Point13_ncep', 'Temperature_Point14_ncep',
       'Temperature_Point15_ncep', 'Temperature_Point16_ncep',
       'Temperature_Point17_ncep', 'Temperature_Point18_ncep',
       'Temperature_Point19_ncep'
]

for dwd_col in dwd_cols:
    ncep_col = dwd_col.replace("dwd", "ncep")
    if ncep_col in df.columns:
        df.loc[mask, dwd_col] = df.loc[mask, ncep_col]

## Features

In [87]:
def pv_temperature_efficiency(irradiance, ambient_temp, NOCT=45, wind_speed=1, eta_0=0.18, beta=0.004):
    """
    Calculate the PV cell temperature and efficiency based on solar irradiance and ambient temperature.
    """
    
    Tc = ambient_temp + (NOCT - 20) * (irradiance / 800)
    
    efficiency = eta_0 * (1 - beta * (Tc - 25))
    
    return Tc, efficiency

In [88]:
temperature_columns = [
    'Temperature_Point0_dwd',
       'Temperature_Point1_dwd', 'Temperature_Point2_dwd',
       'Temperature_Point3_dwd', 'Temperature_Point4_dwd',
       'Temperature_Point5_dwd', 'Temperature_Point6_dwd',
       'Temperature_Point7_dwd', 'Temperature_Point8_dwd',
       'Temperature_Point9_dwd', 'Temperature_Point10_dwd',
       'Temperature_Point11_dwd', 'Temperature_Point12_dwd',
       'Temperature_Point13_dwd', 'Temperature_Point14_dwd',
       'Temperature_Point15_dwd', 'Temperature_Point16_dwd',
       'Temperature_Point17_dwd', 'Temperature_Point18_dwd',
       'Temperature_Point19_dwd', 'Temperature_Point0_ncep',
       'Temperature_Point1_ncep', 'Temperature_Point2_ncep',
       'Temperature_Point3_ncep', 'Temperature_Point4_ncep',
       'Temperature_Point5_ncep', 'Temperature_Point6_ncep',
       'Temperature_Point7_ncep', 'Temperature_Point8_ncep',
       'Temperature_Point9_ncep', 'Temperature_Point10_ncep',
       'Temperature_Point11_ncep', 'Temperature_Point12_ncep',
       'Temperature_Point13_ncep', 'Temperature_Point14_ncep',
       'Temperature_Point15_ncep', 'Temperature_Point16_ncep',
       'Temperature_Point17_ncep', 'Temperature_Point18_ncep',
       'Temperature_Point19_ncep',
]

irradiance_columns = [
    'SolarDownwardRadiation_Point0_dwd', 'SolarDownwardRadiation_Point1_dwd',
    'SolarDownwardRadiation_Point2_dwd', 'SolarDownwardRadiation_Point3_dwd',
    'SolarDownwardRadiation_Point4_dwd', 'SolarDownwardRadiation_Point5_dwd',
    'SolarDownwardRadiation_Point6_dwd', 'SolarDownwardRadiation_Point7_dwd',
    'SolarDownwardRadiation_Point8_dwd', 'SolarDownwardRadiation_Point9_dwd',
    'SolarDownwardRadiation_Point10_dwd', 'SolarDownwardRadiation_Point11_dwd',
    'SolarDownwardRadiation_Point12_dwd', 'SolarDownwardRadiation_Point13_dwd',
    'SolarDownwardRadiation_Point14_dwd', 'SolarDownwardRadiation_Point15_dwd',
    'SolarDownwardRadiation_Point16_dwd', 'SolarDownwardRadiation_Point17_dwd',
    'SolarDownwardRadiation_Point18_dwd', 'SolarDownwardRadiation_Point19_dwd',
    'SolarDownwardRadiation_Point0_ncep', 'SolarDownwardRadiation_Point1_ncep',
    'SolarDownwardRadiation_Point2_ncep', 'SolarDownwardRadiation_Point3_ncep',
    'SolarDownwardRadiation_Point4_ncep', 'SolarDownwardRadiation_Point5_ncep',
    'SolarDownwardRadiation_Point6_ncep', 'SolarDownwardRadiation_Point7_ncep',
    'SolarDownwardRadiation_Point8_ncep', 'SolarDownwardRadiation_Point9_ncep',
    'SolarDownwardRadiation_Point10_ncep', 'SolarDownwardRadiation_Point11_ncep',
    'SolarDownwardRadiation_Point12_ncep', 'SolarDownwardRadiation_Point13_ncep',
    'SolarDownwardRadiation_Point14_ncep', 'SolarDownwardRadiation_Point15_ncep',
    'SolarDownwardRadiation_Point16_ncep', 'SolarDownwardRadiation_Point17_ncep',
    'SolarDownwardRadiation_Point18_ncep', 'SolarDownwardRadiation_Point19_ncep'
]

for i in range(20):
    for source in ['dwd', 'ncep']:
        temp_col = f'Temperature_Point{i}_{source}'
        irradiance_col = f'SolarDownwardRadiation_Point{i}_{source}'
        panel_temp_col = f'Panel_Temperature_Point{i}_{source}'
        panel_eff_col = f'Panel_Efficiency_Point{i}_{source}'
        
        df[panel_temp_col], df[panel_eff_col] = zip(*df.apply(
            lambda row: pv_temperature_efficiency(row[irradiance_col], row[temp_col]), axis=1))

In [89]:
df["SolarDownwardRadiation_Mean"] = df.filter(like= "SolarDownwardRadiation").mean(axis= 1)
df["SolarDownwardRadiation_ncep_Mean"] = df.filter(regex= r"SolarDownwardRadiation.*_ncep").mean(axis= 1)
df["SolarDownwardRadiation_dwd_Mean"] = df.filter(regex= r"SolarDownwardRadiation.*_dwd").mean(axis= 1)
df["SolarDownwardRadiation_RW_Mean_1h"] = df["SolarDownwardRadiation_Mean"].rolling(window= 2).mean()
df["SolarDownwardRadiation_RW_dwd_Mean_30min"] = df["SolarDownwardRadiation_dwd_Mean"].rolling(window= 1).mean()
df["SolarDownwardRadiation_RW_dwd_Mean_1h"] = df["SolarDownwardRadiation_dwd_Mean"].rolling(window= 2).mean()
df["SolarDownwardRadiation_dwd_Mean_Lag_30min"] = df["SolarDownwardRadiation_dwd_Mean"].shift(periods= 1)
df["SolarDownwardRadiation_Mean_Lag_1h"] = df["SolarDownwardRadiation_Mean"].shift(periods= 2)
df["SolarDownwardRadiation_Mean_Lag_24h"] = df["SolarDownwardRadiation_Mean"].shift(periods= 48)
df["Panel_Efficiency_Mean"] = df.filter(regex= r"Panel_Efficiency.*").mean(axis= 1)
df["Panel_Efficiency_Std"] = df.filter(regex= r"Panel_Efficiency.*").std(axis= 1)
df["Panel_Temperature_Mean"] = df.filter(regex= r"Panel_Temperature.*").mean(axis= 1)
df["Panel_Temperature_Std"] = df.filter(regex= r"Panel_Temperature.*").std(axis= 1)
df["Temperature_dwd_Std"] = df.filter(regex= r"Temperature.*_dwd").std(axis= 1)
df["Temperature_dwd_Mean"] = df.filter(regex= r"Temperature.*_dwd").mean(axis= 1)
df["sin_hour"] = np.sin(2 * np.pi * df["valid_time"].dt.hour / 24)
df["cos_hour"] = np.cos(2 * np.pi * df["valid_time"].dt.hour / 24)
df["sin_day"] = np.sin(2 * np.pi * df["valid_time"].dt.dayofyear / 365)
df["cos_day"] = np.cos(2 * np.pi * df["valid_time"].dt.dayofyear / 365)
df["Solar_MWh_Lag_48h"] = df["Solar_MWh_credit"].shift(periods= 96)
df["Capacity_MWP_Lag_48h"] = df["capacity_mwp"].shift(periods= 96)

/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_71710/3364608497.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["sin_day"] = np.sin(2 * np.pi * df["valid_time"].dt.dayofyear / 365)
/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_71710/3364608497.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["cos_day"] = np.cos(2 * np.pi * df["valid_time"].dt.dayofyear / 365)
/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_71710/3364608497.py:20: PerformanceWarning: DataFrame is highly fragmente

In [90]:
df

,timestamp_utc,generation_mw,installed_capacity_mwp,capacity_mwp,reference_time,valid_time,CloudCover_Point0_dwd,CloudCover_Point1_dwd,CloudCover_Point2_dwd,CloudCover_Point3_dwd,...,Panel_Temperature_Mean,Panel_Temperature_Std,Temperature_dwd_Std,Temperature_dwd_Mean,sin_hour,cos_hour,sin_day,cos_day,Solar_MWh_Lag_48h,Capacity_MWP_Lag_48h
0,2020-09-21 00:00:00+00:00,0.0,2228.460880,2130.749513,2020-09-20 06:00:00+00:00,2020-09-21 00:00:00+00:00,0.628613,0.620742,0.397246,0.655645,...,12.344903,0.968971,0.745928,11.983773,0.000000,1.000000,-0.988678,-0.150055,NaN,NaN
1,2020-09-21 00:30:00+00:00,0.0,2228.460880,2130.749513,2020-09-20 06:00:00+00:00,2020-09-21 00:30:00+00:00,0.569268,0.611611,0.318320,0.484082,...,12.238025,0.986570,0.767935,11.885430,0.000000,1.000000,-0.988678,-0.150055,NaN,NaN
2,2020-09-21 01:00:00+00:00,0.0,2228.460880,2130.749513,2020-09-20 06:00:00+00:00,2020-09-21 01:00:00+00:00,0.509922,0.602480,0.239395,0.312520,...,12.131147,1.010593,0.805364,11.787464,0.258819,0.965926,-0.988678,-0.150055,NaN,NaN
3,2020-09-21 01:30:00+00:00,0.0,2228.460880,2130.749513,2020-09-20 06:00:00+00:00,2020-09-21 01:30:00+00:00,0.525977,0.619951,0.119697,0.445312,...,12.001353,1.037686,0.860042,11.620973,0.258819,0.965926,-0.988678,-0.150055,NaN,NaN
4,2020-09-21 02:00:00+00:00,0.0,2228.460880,2130.749513,2020-09-20 06:00:00+00:00,2020-09-21 02:00:00+00:00,0.542031,0.637422,0.000000,0.578105,...,11.871558,1.074863,0.929794,11.454850,0.500000,0.866025,-0.988678,-0.150055,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63568,2024-05-19 21:30:00+00:00,0.0,2741.831037,2573.542092,2024-05-18 06:00:00+00:00,2024-05-19 21:30:00+00:00,0.000576,0.285166,0.000000,0.187998,...,10.272989,0.966369,0.603127,10.764829,-0.707107,0.707107,0.668064,-0.744104,0.0,2573.662184
63569,2024-05-19 22:00:00+00:00,0.0,2741.831037,2573.542092,2024-05-18 06:00:00+00:00,2024-05-19 22:00:00+00:00,0.001152,0.335547,0.000000,0.276719,...,9.897726,0.917137,0.485690,10.364293,-0.500000,0.866025,0.668064,-0.744104,0.0,2573.662184
63570,2024-05-19 22:30:00+00:00,0.0,2741.831037,2573.542092,2024-05-18 06:00:00+00:00,2024-05-19 22:30:00+00:00,0.070967,0.421611,0.110117,0.359990,...,9.624246,0.916183,0.400042,10.080946,-0.500000,0.866025,0.668064,-0.744104,0.0,2573.662184
63571,2024-05-19 23:00:00+00:00,0.0,2741.831037,2573.542092,2024-05-18 06:00:00+00:00,2024-05-19 23:00:00+00:00,0.140781,0.507676,0.220234,0.443262,...,9.350766,0.936262,0.340238,9.798229,-0.258819,0.965926,0.668064,-0.744104,0.0,2573.662184


In [91]:
features = [
    "SolarDownwardRadiation_Mean",
    "SolarDownwardRadiation_RW_Mean_1h",
    "SolarDownwardRadiation_RW_dwd_Mean_30min",
    "SolarDownwardRadiation_dwd_Mean_Lag_30min",
    "SolarDownwardRadiation_Mean_Lag_1h",
    "SolarDownwardRadiation_Mean_Lag_24h",
    "Panel_Efficiency_Mean",        
    "Panel_Efficiency_Std",
    "Panel_Temperature_Mean",
    "Panel_Temperature_Std",
    "Temperature_dwd_Std",
    "Temperature_dwd_Mean",
    "cos_hour",
    "cos_day",
    "Capacity_MWP_Lag_48h",
    "Solar_MWh_Lag_48h"
    ]

df_test = df[features + ["Solar_MWh_credit"]]

In [92]:
df_test.columns

Index(['SolarDownwardRadiation_Mean', 'SolarDownwardRadiation_RW_Mean_1h',
       'SolarDownwardRadiation_RW_dwd_Mean_30min',
       'SolarDownwardRadiation_dwd_Mean_Lag_30min',
       'SolarDownwardRadiation_Mean_Lag_1h',
       'SolarDownwardRadiation_Mean_Lag_24h', 'Panel_Efficiency_Mean',
       'Panel_Efficiency_Std', 'Panel_Temperature_Mean',
       'Panel_Temperature_Std', 'Temperature_dwd_Std', 'Temperature_dwd_Mean',
       'cos_hour', 'cos_day', 'Capacity_MWP_Lag_48h', 'Solar_MWh_Lag_48h',
       'Solar_MWh_credit'],
      dtype='object')

In [93]:
df_test.to_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/Generation_forecast/Solar_forecast/data/train1.csv", index= False)

## Target Norm Capacity

In [94]:
df

,timestamp_utc,generation_mw,installed_capacity_mwp,capacity_mwp,reference_time,valid_time,CloudCover_Point0_dwd,CloudCover_Point1_dwd,CloudCover_Point2_dwd,CloudCover_Point3_dwd,...,Panel_Temperature_Mean,Panel_Temperature_Std,Temperature_dwd_Std,Temperature_dwd_Mean,sin_hour,cos_hour,sin_day,cos_day,Solar_MWh_Lag_48h,Capacity_MWP_Lag_48h
0,2020-09-21 00:00:00+00:00,0.0,2228.460880,2130.749513,2020-09-20 06:00:00+00:00,2020-09-21 00:00:00+00:00,0.628613,0.620742,0.397246,0.655645,...,12.344903,0.968971,0.745928,11.983773,0.000000,1.000000,-0.988678,-0.150055,NaN,NaN
1,2020-09-21 00:30:00+00:00,0.0,2228.460880,2130.749513,2020-09-20 06:00:00+00:00,2020-09-21 00:30:00+00:00,0.569268,0.611611,0.318320,0.484082,...,12.238025,0.986570,0.767935,11.885430,0.000000,1.000000,-0.988678,-0.150055,NaN,NaN
2,2020-09-21 01:00:00+00:00,0.0,2228.460880,2130.749513,2020-09-20 06:00:00+00:00,2020-09-21 01:00:00+00:00,0.509922,0.602480,0.239395,0.312520,...,12.131147,1.010593,0.805364,11.787464,0.258819,0.965926,-0.988678,-0.150055,NaN,NaN
3,2020-09-21 01:30:00+00:00,0.0,2228.460880,2130.749513,2020-09-20 06:00:00+00:00,2020-09-21 01:30:00+00:00,0.525977,0.619951,0.119697,0.445312,...,12.001353,1.037686,0.860042,11.620973,0.258819,0.965926,-0.988678,-0.150055,NaN,NaN
4,2020-09-21 02:00:00+00:00,0.0,2228.460880,2130.749513,2020-09-20 06:00:00+00:00,2020-09-21 02:00:00+00:00,0.542031,0.637422,0.000000,0.578105,...,11.871558,1.074863,0.929794,11.454850,0.500000,0.866025,-0.988678,-0.150055,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63568,2024-05-19 21:30:00+00:00,0.0,2741.831037,2573.542092,2024-05-18 06:00:00+00:00,2024-05-19 21:30:00+00:00,0.000576,0.285166,0.000000,0.187998,...,10.272989,0.966369,0.603127,10.764829,-0.707107,0.707107,0.668064,-0.744104,0.0,2573.662184
63569,2024-05-19 22:00:00+00:00,0.0,2741.831037,2573.542092,2024-05-18 06:00:00+00:00,2024-05-19 22:00:00+00:00,0.001152,0.335547,0.000000,0.276719,...,9.897726,0.917137,0.485690,10.364293,-0.500000,0.866025,0.668064,-0.744104,0.0,2573.662184
63570,2024-05-19 22:30:00+00:00,0.0,2741.831037,2573.542092,2024-05-18 06:00:00+00:00,2024-05-19 22:30:00+00:00,0.070967,0.421611,0.110117,0.359990,...,9.624246,0.916183,0.400042,10.080946,-0.500000,0.866025,0.668064,-0.744104,0.0,2573.662184
63571,2024-05-19 23:00:00+00:00,0.0,2741.831037,2573.542092,2024-05-18 06:00:00+00:00,2024-05-19 23:00:00+00:00,0.140781,0.507676,0.220234,0.443262,...,9.350766,0.936262,0.340238,9.798229,-0.258819,0.965926,0.668064,-0.744104,0.0,2573.662184


In [95]:
import numpy as np

df["Target_Capacity_MWP_%"] = np.where(df["Solar_MWh_credit"] == 0.0, 0, df["Solar_MWh_credit"] / df["capacity_mwp"])
df["Target_Capacity_MWP_%_Lag_48"] = df["Target_Capacity_MWP_%"].shift(96)

/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_71710/3338083059.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["Target_Capacity_MWP_%"] = np.where(df["Solar_MWh_credit"] == 0.0, 0, df["Solar_MWh_credit"] / df["capacity_mwp"])
/var/folders/fm/5lbdpfj928d6sqhy_19cp6ww0000gn/T/ipykernel_71710/3338083059.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["Target_Capacity_MWP_%_Lag_48"] = df["Target_Capacity_MWP_%"].shift(96)


In [96]:
df["capacity_mwp"].mean() # Capacity to later calc absolute Loss

np.float64(2281.9884164450805)

In [97]:
features = [
    "SolarDownwardRadiation_Mean",
    "SolarDownwardRadiation_RW_Mean_1h",
    "SolarDownwardRadiation_RW_dwd_Mean_30min",
    "SolarDownwardRadiation_dwd_Mean_Lag_30min",
    "SolarDownwardRadiation_Mean_Lag_1h",
    "SolarDownwardRadiation_Mean_Lag_24h",
    "Panel_Efficiency_Mean",
    "Panel_Efficiency_Std",
    "Panel_Temperature_Mean",
    "Panel_Temperature_Std",
    "Temperature_dwd_Std",
    "Temperature_dwd_Mean",
    "cos_hour",
    "cos_day",
    "Solar_MWh_Lag_48h",
    "Capacity_MWP_Lag_48h",
    "Target_Capacity_MWP_%_Lag_48",
    "Target_Capacity_MWP_%"
    ]

df_test_norm = df[features]

In [98]:
df_test_norm

,SolarDownwardRadiation_Mean,SolarDownwardRadiation_RW_Mean_1h,SolarDownwardRadiation_RW_dwd_Mean_30min,SolarDownwardRadiation_dwd_Mean_Lag_30min,SolarDownwardRadiation_Mean_Lag_1h,SolarDownwardRadiation_Mean_Lag_24h,Panel_Efficiency_Mean,Panel_Efficiency_Std,Panel_Temperature_Mean,Panel_Temperature_Std,Temperature_dwd_Std,Temperature_dwd_Mean,cos_hour,cos_day,Solar_MWh_Lag_48h,Capacity_MWP_Lag_48h,Target_Capacity_MWP_%_Lag_48,Target_Capacity_MWP_%
0,0.002455,NaN,0.004910,NaN,NaN,NaN,0.189112,0.000698,12.344903,0.968971,0.745928,11.983773,1.000000,-0.150055,NaN,NaN,NaN,0.0
1,0.000134,0.001295,0.000269,0.004910,NaN,NaN,0.189189,0.000710,12.238025,0.986570,0.767935,11.885430,1.000000,-0.150055,NaN,NaN,NaN,0.0
2,-0.002187,-0.001026,-0.004373,0.000269,0.002455,NaN,0.189266,0.000728,12.131147,1.010593,0.805364,11.787464,0.965926,-0.150055,NaN,NaN,NaN,0.0
3,0.001375,-0.000406,0.002750,-0.004373,0.000134,NaN,0.189359,0.000747,12.001353,1.037686,0.860042,11.620973,0.965926,-0.150055,NaN,NaN,NaN,0.0
4,0.004936,0.003156,0.009872,0.002750,-0.002187,NaN,0.189452,0.000774,11.871558,1.074863,0.929794,11.454850,0.866025,-0.150055,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63568,-0.003922,-0.001734,-0.007845,0.000909,-0.004033,-0.003981,0.190603,0.000696,10.272989,0.966369,0.603127,10.764829,0.707107,-0.744104,0.0,2573.662184,0.0,0.0
63569,-0.008299,-0.006111,-0.016599,-0.007845,0.000455,-0.030075,0.190874,0.000660,9.897726,0.917137,0.485690,10.364293,0.866025,-0.744104,0.0,2573.662184,0.0,0.0
63570,0.005495,-0.001402,0.010989,-0.016599,-0.003922,-0.005774,0.191071,0.000660,9.624246,0.916183,0.400042,10.080946,0.866025,-0.744104,0.0,2573.662184,0.0,0.0
63571,0.019289,0.012392,0.038577,0.010989,-0.008299,0.018527,0.191267,0.000674,9.350766,0.936262,0.340238,9.798229,0.965926,-0.744104,0.0,2573.662184,0.0,0.0


In [99]:
df_test_norm.to_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/Generation_forecast/Solar_forecast/data/train_norm1.csv", index= False)